# Matrix Multiply

The following sequence of cells illustrate hardware processing of a matrix multiply computation:
<div align="center"  style="font-size:150%">
    <br>
    <i> Z<sub>m,n</sub> = A<sub>m,k</sub> x B<sub>k,n</sub></i>
</div>

## Exercise 02.2.1 Sparse Matrix Multiplitcation
<img align="center" src="figures/02.2.1.spMspM_setup.png" alt="spMspM_setup" style="width:100%">

### Understanding the inputs: Problem Specification

In [ ]:
%set_env BASELINE_DIR=../../../example_designs
%set_env DESIGNS_DIR=../../../example_designs/example_designs/sparseloop
%set_env SCRIPTS_DIR=../scripts

In [ ]:
%%bash
cd $DESIGNS_DIR/02.2.1-spMspM/ && cat problem.yaml

### Understanding the inputs: Architecture Specification
We consider a more realistic architecture with a larger `BackingStorage` and a smaller local `Buffer`. We use small storages here for simplicity.

In [ ]:
%%bash
cd $DESIGNS_DIR/02.2.1-spMspM/ && cat arch.yaml

### Understanding the inputs: Mapping Specification
We need to specify the loops for each storage unit. Since we are doing simple streaming from `BackingStorage` to `Buffer`, all the loops below the `BackingStorage` level have trivial bounds.

In [ ]:
%%bash
cd $DESIGNS_DIR/02.2.1-spMspM/ && cat mapping.yaml

### Understanding the inputs: Sparse Optimization Feature Specification
As shown in the setup figure above, we need the following sparse optimizations to achieve the expected behaviors
- compressed representation of tensors A and B at both storage levels
- skipping optimization applied to all A, B, and Z with different skipping conditions

In [ ]:
%%bash
cd $DESIGNS_DIR/02.2.1-spMspM/ && cat arch.yaml

### Run Example

In [ ]:
%%bash
cd $BASELINE_DIR
python3 run_example_designs.py --architecture sparseloop/02.2.1-spMspM

### Examine Important Stats

1. `A` Tensor Related: 
  - Reduced data storage capacity requirement due to its compressed data representation.
  - There are skipped data accesses due to the explicit skipping optimization of A based on B, i.e., if B has an empty payload value, then it is not necessary to read out the corresponding A anymore.
  - Reduced number of accesses actual data accesses because of compressed representation format as well as explicit skipping optimization.
  - Metadata storage overhead and accesses. Note that # of metadata units > # of nonzero values in A. There are 25 units of metadata overhead because we have 16 coordinates in the lower rank and 8+1 offsets in the upper rank. Thus, the total number of metadata + number of data units < original tile shape.
  - Some of the metadata accesses are also skipped.
2. `B` Tensor Related
  - Reduced data storage capacity requirement due to its compressed data representation.
  - Skipped data accesses due to the explicit skipping optimization of B based on A, i.e., if A has an empty payload value, then it is not necessary to read out the corresponding B fiber.
  - Reduced number of actual data accesses because of compressed representation format as well as explicit skipping optimization.
  - Metadata storage overhead and accesses. Note the due to the higher density in B, total number of metadata + number of data units > original tile shape because of higher tensor density.
3. `Z`: 
  - Capacity requirement == tile shape due to uncompressed representation.
  - Reduced number of actual reads and actual updates, and skipped reads && skipped updates counts show up.
4. `MAC`: reduced number of actual computes since the operands are skipped at the `Buffer` and  become `NOT_EXSIT` at the compute.
6. Total number of cycles reduced as both computes and storage accesses are skipped. As a result, MAC has a full utilization fo 1.0.
7. Total energy reduced due to reduced storage accesses and computes.

In [ ]:
%set_env OUT_DIR=../../../example_designs/example_designs/sparseloop/02.2.1-spMspM/outputs/default_problem

In [ ]:
%%bash
export SCRIPT_PATH=$(realpath $SCRIPTS_DIR/02.2.1-spMspM-aggregated-stats.sh)
chmod +x $SCRIPT_PATH && cd $OUT_DIR && $SCRIPT_PATH

## Exercise 02.2.2 Tiled Sparse Matrix Multiplitcation

### Understanding  Inputs: Mapping Specification with Tiling on `N`

In [ ]:
%%bash
cd $DESIGNS_DIR/02.2.2-spMspM-tiled/ && cat mapping.yaml

### Understanding Inputs: Architecture with A Smaller Buffer

Tiling allows smaller `B` and `Z` tiles to be stored in the `Buffer`, so we can reduce the storage space in our architecture. In this specific example, we reduce the `depth` of the `Buffer` to half, i.e., 128 to 64.

In [ ]:
%%bash
echo "==========================================="
echo "     Untiled Exampe Buffer Specification"
echo "==========================================="

cd $DESIGNS_DIR/02.2.1-spMspM
grep Buffer -A 9 arch.yaml

printf "\n===========================================\n"
printf "    Tiled Exampe Buffer Specification\n"
printf "===========================================\n"

cd ../02.2.2-spMspM-tiled
grep Buffer -A 9 arch.yaml

### Understanding Inputs: Data Representaiton Format Specification for Pretiled Tensors

#### Tiling introduces splitted rank(s) and thus additional ranks in tensor
<img align="center" src="figures/02.2.2.dense_tiled_mapping.png" alt="tiled_mapping" style="width:70%">


#### Pretiling 
Accelerator designs usually only support concordant traversal of compressed tensors, and thus when tiling is necessary, the input tensor to the accelerator is often **pre-tiled**.

Thus the `B` tensor stored in `BackingStorage` needs to have three ranks, N1, K, N0 *(N/N1)*. Failing to specify enough ranks for the tensor representation will result in an error.

In this example, the top `N1` rank is only useful in `BackingStroage` to identify the tiles (elements in the `N1` ranks) being transferred to `Buffer`, so the `N1` rank is not transferred to `Buffer` storage.

In [ ]:
%%bash
cd $DESIGNS_DIR/02.2.2-spMspM-tiled && grep sparse_optimizations -A 9 arch.yaml

### Run Example

In [ ]:
%%bash
cd $BASELINE_DIR
python3 run_example_designs.py --architecture sparseloop/02.2.2-spMspM-tiled

### Examine Important Stats: `B` Tensor - Centric View
- `A` Tensor Related: 
  - All observations from the last part stay true, as `A` tensor is not tiled.
- `B` Tensor Related
  - `BackingStorage` has the tensor pre-tiled to 3 ranks. 
  - Metadata storage capacity overhead increased significantly in `BackingStorage`, and thus the number of metadata reads.
  - Smaller tiles are stored in `Buffer`, each tile in `Buffer` is still in rank2 format.
- `Z` Tensor Related: 
  - Smaller tiles are stored in `Buffer` in uncompressed format.
  - Other observations stay the same as the untiled case.
- `MAC`: Same behaviors as untiled.

In [ ]:
%set_env OUT_DIR=../../../example_designs/example_designs/sparseloop/02.2.2-spMspM-tiled/outputs/default_problem

In [ ]:
%%bash
export SCRIPT_PATH=$(realpath $SCRIPTS_DIR/02.2.2-spMspM-pretiling-stats.sh)
chmod +x $SCRIPT_PATH && cd $OUT_DIR && $SCRIPT_PATH

### Compare Untiled and Tiled Examples
Compared to untiled example, the tiled example consumes 67% energy per algorithmic compute.

In [ ]:
%%bash
echo "========================================"
echo "             Untiled Run                "
echo "========================================"
cd $BASELINE_DIR
python3 run_example_designs.py --architecture sparseloop/02.2.1-spMspM

echo "========================================"
echo "               Tiled Run                "
echo "========================================"
python3 run_example_designs.py --architecture sparseloop/02.2.2-spMspM-tiled